In [2]:
DEVICE = "cpu"

In [3]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

tensor([[24.5701, 19.3049]], grad_fn=<TBackward0>)
tensor([[0.9949, 0.0051]], grad_fn=<SoftmaxBackward0>)


In [3]:
import torch
import importlib
import loaders
import functools
import edit
import importlib
import taming.modules.vqvae.quantize
importlib.reload( taming.modules.vqvae.quantize)
importlib.reload(loaders)
from loaders import load_default

model = load_default(DEVICE).to(DEVICE)

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


In [10]:
x = list(model.parameters())[0]
DEVICE

'cpu'

In [13]:
from latent_utils import *
# x = get_latent_from_path("./women_aligned/1.png", model, device=DEVICE)
latent = get_latent_from_path("./test_data/face.jpeg", model, device=DEVICE)
# y = get_latent_from_path("./women_aligned/2.png", model)
# im_x = show_latent(model, x)
# im_y = show_latent(model, y)

/opt/homebrew/Caskroom/miniforge/base/envs/taming/lib/python3.8/site-packages/torchvision/transforms/functional.py:442: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# inputs = processor(text=["a photo of a man", "a photo of a woman"], images=im_x, return_tensors="pt", padding=True)

# outputs = clip(**inputs)
# logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities
# target = torch.Tensor([1, 0])
# print(logits_per_image)
# print(logits_per_image.shape)
# # print(probs)

In [ ]:
target = torch.Tensor([1, 0])

In [ ]:
tvec = torch.randn_like(latent)
show_latent(model, latent + tvec)

In [ ]:
print(target)
list(model.parameters())[0]

In [ ]:
import torch.nn as nn
from img_processing import *
class TensorExtractor(nn.Module):
    def __init__(self, latent) -> None:
        super().__init__()
        #TODO tighter distribution
        self.latent = latent
        self.loss_fn = torch.nn.BCELoss()
        self.logits_loss = torch.nn.BCEWithLogitsLoss()
        self.optim = torch.optim.Adam([self.vector])
    def forward(self):
        trans_latent = self.latent + self.vector
        dec = model.decode(trans_latent.to(DEVICE))
        trans_im = custom_to_pil(dec[0])
        return trans_im, self.vector

def train(model):
    latent = get_latent_from_path("./test_data/face.jpeg", model)
    vector = torch.randn_like(latent)
    optim = torch.optim.Adam([self.vector])
    for i in range 100:
        trans_im, vector = 
        inputs = processor(text=["a photo of a man", "a photo of a woman"], images=trans_im, return_tensors="pt", padding=True)
        outputs = clip(**inputs)
        logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
        probs = logits_per_image.softmax(dim=1) 
        self.optim.zero_grad()
        loss = self.logits_loss(logits_per_image, target) 
        loss.backward()



        



In [5]:
import torch.nn as nn
from img_processing import *
from utils import freeze_module
import matplotlib.pyplot as plt
from tqdm import tqdm
class TensorExtractor(nn.Module):
    def __init__(self, latent, model) -> None:
        super().__init__()
        #TODO tighter distribution
        self.latent = latent
        model.eval()
        # freeze_module(model)
        self.model = model
        self.model.to_torchscript(method="trace", example_inputs=torch.ones((1, 3, 256, 256)))
        self.loss_fn = torch.nn.BCELoss()
        self.logits_loss = torch.nn.BCEWithLogitsLoss()
    def forward(self, vector):
        trans_latent = self.latent + vector
        dec = self.model.decode(trans_latent.to(DEVICE))
        # trans_im = custom_to_pil(dec[0])
        return dec.squeeze()
    def optimize(self):
        vector = torch.randn_like(latent, requires_grad=True)
        target = torch.tensor([[1., 0.]])
        self.optim = torch.optim.Adam([vector])
        intermediate = []
        for i in tqdm(range(40)):
            self.optim.zero_grad()
            trans_im = self(vector)
            # if i % 10 == 0: 
            #     intermediate.append(trans_im)
            #     plt.figure(figsize=(3, 3))
            #     plt.imshow(custom_to_pil(trans_im[0]))
            #     plt.show()
            inputs = processor(text=["a photo of a man", "a photo of a woman"], images=trans_im, return_tensors="pt", padding=True)
            
            outputs = clip(**inputs)
            logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
            print(torch.autograd.grad(torch.mean(outputs.logits_per_image), inputs))
            # probs = logits_per_image.softmax(dim=1) 
            print(logits_per_image)
            loss = self.logits_loss(logits_per_image, target) 
            print(loss)
            loss.backward()
            self.optim.step()
        show_latent(self.model, self.latent + vector)
        return vector

ex = TensorExtractor(latent, model).to(DEVICE)
vec = ex.optimize()


# from torch.utils.tensorboard import SummaryWriter

# # default `log_dir` is "runs" - we'll be more specific here
# writer = SummaryWriter('runs/ex')
# tvec = torch.randn_like(latent)
# writer.add_graph(ex, tvec)
# writer.close()


        



NameError: name 'latent' is not defined

In [ ]:

vec

In [ ]:
target = torch.tensor([[1., 0.]])

In [ ]:
target.dtype

In [ ]:
latent.shape

In [ ]:
model.eval()
x = next(model.parameters())
x

In [ ]:
t.shape

In [ ]:
vec = torch.randn_like(latent)
op = torch.optim.Adam([vec])
print(op)